# Config

In [2]:
# ___Names&Paths setup___
driveFolder = 'Minecraft-server'
worldName = 'KSworld'

# Please, make an archive of the whole server on the driveFolder for easy transfer
serverArchiveName = 'MC'

# Should be the same name as the drive one
kaggleserver_folder = '/kaggle/working/' + driveFolder
Overworld = worldName
Nether = worldName + '_nether'
End = worldName + '_the_end'

# Check the command in run.sh
forgeCommand = "@libraries/net/minecraftforge/forge/1.18.2-40.2.10/unix_args.txt"

TempArea = '/kaggle/working/transferArea/'

# Initialization

In [4]:
import shutil, zipfile, os, re, json, threading, time
from datetime import datetime
! pip install pydrive2 &>/dev/null && echo "pydrive2 installed" || echo "Failed to install pydrive2"
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
!pip install mcrcon &>/dev/null && echo "MCRcon installed" || echo "Failed to install MCRcon"
from mcrcon import MCRcon
import subprocess, multiprocessing
import ipywidgets as widgets
from IPython.display import display, clear_output

# Move gdrive Credentials from input dataset to transfer folder
%mkdir {TempArea}
shutil.copy("/kaggle/input/credits/client_secrets.json",TempArea)
shutil.copy("/kaggle/input/credits/gdrive.txt",TempArea)

%cd {TempArea}
gauth = GoogleAuth()
gauth.LoadCredentialsFile("gdrive.txt")
drive = GoogleDrive(gauth)


# Upload function
def upload(file):
    nfile = file
    if file == driveFolder:
        file = serverArchiveName
        locate = '/kaggle/working/'
    else:
        locate = kaggleserver_folder
    # Zip the folder before transfer
    shutil.make_archive(TempArea + file, 'zip', locate, nfile)

    # Get the ID of the drive folder where we'll upload
    folderid = drive.ListFile({'q': f"title = '{driveFolder}' and trashed=false"}).GetList()[0]['id']
    try: 
        # Check if the current file we're working with already exists on the drive
        Savingid = drive.ListFile({'q': f"title = '{file + '.zip'}' and trashed=false"}).GetList()[0]['id']
        # if so, overwrite it
        Saving = drive.ListFile({'q': f"title = '{file + '.zip'}' and trashed=false"}).GetList()[0]
        Saving.SetContentFile(TempArea + file + '.zip')
    except IndexError as nothere:
        # if not, create it
        Saving = drive.CreateFile({'title': f"{file + '.zip'}",'parents': [{'id': folderid}]})
        Saving.SetContentFile(TempArea + file + '.zip')
    Saving.Upload()

# Auto-saving function
def run_saveToDrive_task():
    while Running == True:
        Timestamp = datetime.now().strftime("%H:%M:%S")
        print(f"[{Timestamp}] [Auto-save] Saving World & logs to Drive")
        upload(Overworld)
        upload(Nether)
        upload(End)
        upload('logs')
        i = 0
        Frequency = 300 # Wait 5 minutes
        while i <= Frequency:
            if Running == True:
                time.sleep(10)
                i += 10
            else:
                break

# Launching server function
def run_command():
    command = f'java {memory_allocation} {server_flags} {forgeCommand} nogui'
    process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

    # Put the output to a widget
    for line in process.stdout:
        with output_widget:
            print(line, end='')
        
# Create widget to show outputs
output_widget = widgets.Output()

# Download function
def download(file):
    if file == serverArchiveName:
        location = '/kaggle/working/'
    else:
        location = kaggleserver_folder
    importingid = drive.ListFile({'q': f"title = '{file + '.zip'}' and trashed=false"}).GetList()[0]['id']
    importing = drive.CreateFile({'id': importingid})
    importing.GetContentFile(file + '.zip')
    archive = file + '.zip'
    with zipfile.ZipFile(archive, 'r') as zip_ref :
        zip_ref.extractall(location)

        
download('MC')
print('Importing server done.')
try :
    download(Overworld)
    print('Importing Overworld done.')
    download(Nether)
    print('Importing Nether done.')
    download(End)
    print('Importing End done.')
except FileNotFoundError as NoWorldYet:
    print('No world to import yet.')



updated_lines = []
with open(kaggleserver_folder + 'server.properties', 'r') as file:
    for line in file:
        if 'level-name' in line:
            updated_lines.append(f'level-name={worldName}\n')
        else:
            updated_lines.append(line)

with open(kaggleserver_folder + 'server.properties', 'w') as file:
    file.writelines(updated_lines)




# ___Prepare the environment___
# Update the package lists
!sudo apt update &>/dev/null && echo "apt cache successfully updated" || echo "apt cache update failed, you might receive stale packages"
# Install OpenJDK 17
# !wget -qO - https://adoptopenjdk.jfrog.io/adoptopenjdk/api/gpg/key/public | sudo apt-key add -
# !sudo add-apt-repository --yes https://adoptopenjdk.jfrog.io/adoptopenjdk/deb/ &>/dev/null || echo "Failed to add repo. Still can be ignored if openjdk17 gets installed."
!sudo apt-get install openjdk-17-jre-headless &>/dev/null && echo "Yay! Openjdk17 has been successfully installed." || echo "Failed to install OpenJdk17."
# Perform java version check
java_ver = !java -version 2>&1 | awk -F[\"\.] -v OFS=. 'NR==1{print $2}'
if java_ver[0] == "17" :
  print("Openjdk17 is working correctly, you are good to go.")
else:
  print("Openjdk17 doesn't seems to be installed or isn't working, falling back to java", java_ver[0], ". You might experience reduced performance. Minecraft 1.17 and above might fail to launch.")

# Change directory to the Minecraft server folder
%cd {kaggleserver_folder}

# Import config file.
if os.path.isfile("colabconfig.json"):
  colabconfig = json.load(open("colabconfig.json"))
else:
  colabconfig = {"server_type": "generic"} # using default, if config doesn't exists.
  json.dump(colabconfig, open("colabconfig.json",'w'))

# Server jar names.
jar_list = {'paper': 'server.jar', 'fabric': 'fabric-server-launch.jar', 'generic': 'server.jar', 'bukkit': 'craftbukkit.jar'}
jar_name = jar_list[colabconfig["server_type"]]

# Java arguments.
if colabconfig["server_type"] == "paper":
  server_flags = "-XX:+UseG1GC -XX:+ParallelRefProcEnabled -XX:MaxGCPauseMillis=200 -XX:+UnlockExperimentalVMOptions -XX:+DisableExplicitGC -XX:+AlwaysPreTouch -XX:G1NewSizePercent=30 -XX:G1MaxNewSizePercent=40 -XX:G1HeapRegionSize=8M -XX:G1ReservePercent=20 -XX:G1HeapWastePercent=5 -XX:G1MixedGCCountTarget=4 -XX:InitiatingHeapOccupancyPercent=15 -XX:G1MixedGCLiveThresholdPercent=90 -XX:G1RSetUpdatingPauseTimePercent=5 -XX:SurvivorRatio=32 -XX:+PerfDisableSharedMem -XX:MaxTenuringThreshold=1 -Dusing.aikars.flags=https://mcflags.emc.gs -Daikars.new.flags=true"
else:
  server_flags = "" # aiker's flags might negatively impact performance on non-paper servers.
memory_allocation = "-Xmx8000M -Xms100M"

# Chose the tunnle service you want to use
# Available options: ngrok, argo, playit
tunnel_service = "playit"
print("Proceeding to use", tunnel_service)

if tunnel_service == "playit":
    ! curl -SsL https://playit-cloud.github.io/ppa/key.gpg | sudo apt-key add -
    ! sudo curl -SsL -o /etc/apt/sources.list.d/playit-cloud.list https://playit-cloud.github.io/ppa/playit-cloud.list
    ! sudo apt update &>/dev/null && sudo apt install playit &>/dev/null && echo "Playit.gg installed" || echo "Failed to install playit"
    print('Starting tunnel')
    os.system('playit > playit.log 2>&1 &')
    time.sleep(3)
    # Output the line with the URL to open the tunnel
    with open('playit.log', 'r') as tunnelfile:
        for _ in range(3):
            tunnelfile.readline()
        third_ligne = tunnelfile.readline()
        print(third_ligne)

                                        
                                        
# _____WIP_____                                        
elif tunnel_service == "ngrok":
  !pip -q install pyngrok
  from pyngrok import conf, ngrok

  # Ask for the ngrok authtoken
  print("Get your authtoken from https://dashboard.ngrok.com/auth")
  import getpass
  authtoken = getpass.getpass()  # input your Ngrok auth token everytime you run the cell or simply replace "getpass.getpass()" with your token in "double quotes"
  ! ngrok authtoken $authtoken # login to ngrok

  # Sets default ngrok region
  conf.get_default().region = 'eu'  # Change this to whichever region you want

  # Connect to ngrok
  url = ngrok.connect(25565, 'tcp')
  print('Your server address is ' + ((str(url).split('"')[1::2])[0]).replace('tcp://', ''))
  !ngrok tcp --domain=apparently-brief-polliwog.ngrok-free.app 80
  print('Starting server...')
  ! java $memory_allocation $server_flags $forgeCommand nogui
  #!java $memory_allocation $server_flags -jar $jar_name nogui #@libraries/net/minecraftforge/forge/1.18.2-40.2.10/unix_args.txt nogui

elif tunnel_service == "argo":
  # Download & make argo executable
  !wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
  !chmod +x cloudflared-linux-amd64
  print('Starting tunnel')
  !./cloudflared-linux-amd64 tunnel --url tcp://127.0.0.1:25565 & java $memory_allocation $server_flags -jar $jar_name nogui
# __________

pydrive2 installed
MCRcon installed
mkdir: cannot create directory ‘/kaggle/working/transferArea/’: File exists
/kaggle/working/transferArea
Importing server done.
Importing Overworld done.
Importing Nether done.
Importing End done.
apt cache successfully updated
Yay! Openjdk17 has been successfully installed.
Openjdk17 is working correctly, you are good to go.
/kaggle/working/Minecraft-server
Proceeding to use playit
curl: /opt/conda/lib/libcurl.so.4: no version information available (required by curl)
OK
Playit.gg installed
Starting tunnel
2023-08-21T22:23:32.507123Z  INFO playit_agent_core::setup_config: generated claim url claim_url=https://playit.gg/claim/5b738e9442



# Start Server

In [69]:
Running = True

# Creating threads
#command_thread = threading.Thread(target=run_command) 
command_process = multiprocessing.Process(target=run_command)
Save_thread = threading.Thread(target=run_saveToDrive_task)

# Starting save thread
Save_thread.start()
# Starting server thread
command_process.start()
#command_thread.start()

[21:28:38] [Auto-save] Saving World & logs to Drive

[21:33:02] [Server thread/WARN] [un.UntamedWilds/]: There's no species provided for the EntityType


# Send commands

In [9]:
# Run to send a command to the server | Recommended to stop the server with stop command
# Enabling RCon in server.properties is requiered

MyCommand = "say Hi"

# Init connexion 
with MCRcon("127.0.0.1", "12345") as mcr:
    # Send command
    response = None
    try:
        response = mcr.command(MyCommand)
    except Exception as e:
        print(f"Error while executing RCON command : {e}")

    # Show response if there is one
    if response is not None:
        print(response)
if MyCommand == "stop":
    # Stop the auto-saving (takes up to 10 sec)
    Running = False
    Save_thread.join()
    command_process.join()

    # Saves a last time after shutting down
    upload(Overworld)
    upload(Nether)
    upload(End)
    upload('logs')
    print("Done.")

Stopping the server



RedirectMissingLocation: Redirected but the response is missing a Location: header.

# Show console outputs

In [ ]:
# Show the outputs | Those can be seen at any time in the Console anyway
try:
    while True:
        display(output_widget)
        stopprinting = input("Type 'y' to stop printing outputs.\n")
        if stopprinting == 'y':
            break
        time.sleep(5)
        clear_output(wait=True)
except KeyboardInterrupt as manualstop:
    print('')

# Manual save

In [5]:
# Manually save the worlds & logs
upload(Overworld)
upload(Nether)
upload(End)
upload('logs')
print("Done.")

Done.


# Global server save

In [6]:
# Saving the whole server to drive in case you changed something
upload('Minecraft-server')
print("done.")

done.


# Misc.

In [70]:
# If for any reason you want to stop auto-saving (takes up to 10 sec)
Running = False
Save_thread.join()

In [ ]:
#Quick file edit area | In case you want to edit something like server.properties
with open('/kaggle/working/abc.txt','r') as file:
    content = file.read()
ncontent = content.replace('test1', 'test3')
with open('/kaggle/working/abc.txt','w') as file:
    file.write(ncontent)